In [ ]:
import json
import os
import cv2
import numpy as np
import random

from custom_utils import *
from amodal_utils import *
from coco_json import initialize_coco_json, save_coco_json, add_to_coco_json
from visualization_utils import visualize_merged_amodal_and_modal, visualize_all_masks
from config import INPUT_PATHS, OUTPUT_PATHS, HYPERPARAMETERS

#### Merged image generation

In [ ]:
def merge_leaf_to_cucumber(cucumber_image, leaf_image, cucumber_mask, position, occlusion_ratio = 0.5):
    # 오이 객체 중심 계산
    cucumber_bbox = get_bbox_from_mask(cucumber_mask)
    # 위치에 따른 좌표 계산
    leaf_location = calculate_leaf_location(cucumber_bbox, position)

    resized_leaf_image = resize_leaf_to_target_ratio(cucumber_mask, leaf_image, leaf_location, occlusion_ratio)
    
    merged_image, leaf_mask = merge_and_crop_leaf(cucumber_image, resized_leaf_image, leaf_location)

    return merged_image, leaf_mask


def synthesize_two_images(cucumber_image_path, 
                          cucumber_mask_path, 
                          leaf_image_path, 
                          position, 
                          occlusion_ratio, 
                          save_dir=None, 
                          global_image_id=0, 
                          target_size=(768, 1024)):
    
    # 오이 이미지와 잎 이미지 로드
    cucumber_image = cv2.imread(cucumber_image_path, cv2.IMREAD_UNCHANGED)
    leaf_image = cv2.imread(leaf_image_path, cv2.IMREAD_UNCHANGED)
    cucumber_mask = cv2.imread(cucumber_mask_path, cv2.IMREAD_GRAYSCALE)
    print(f"Processing leaf image: {os.path.basename(leaf_image_path)}")
    # 이미지를 합성
    merged_image, leaf_mask = merge_leaf_to_cucumber(cucumber_image, leaf_image, cucumber_mask, position, occlusion_ratio)

    # 리사이즈
    resized_image, resized_masks = resize_image_and_masks(
        merged_image, [cucumber_mask, leaf_mask], target_size=target_size)
    amodal_mask, leaf_mask = resized_masks

    # 합성 이미지 저장
    cucumber_image_name = os.path.basename(cucumber_image_path)
    merged_image_name = f"{os.path.splitext(cucumber_image_name)[0]}_merged_{global_image_id:06d}.png"
    resized_image_path = save_image(save_dir, merged_image_name, resized_image)

    return resized_image_path, amodal_mask, leaf_mask

##### amodal mask info (occluder, occuluded)

In [ ]:
def generate_annotation(amodal_mask, modal_mask, global_id, image_id, category_id, occluder_segm=[]):
    amodal_segm = mask_to_polygon(amodal_mask)
    amodal_bbox = get_coco_bbox_from_mask(amodal_mask)
    
    process_mask = modal_mask if modal_mask is not None else amodal_mask  # 보이는 영역이 없으면 전체 영역(amodal_mask) 사용
    area = float(np.sum(process_mask == 255))
    segmentation = visible_segm = mask_to_polygon(process_mask)
    visible_bbox = get_coco_bbox_from_mask(process_mask)

    annotation = {
        "id": global_id,
        "image_id": image_id,
        "category_id": category_id,
        "iscrowd": 0,
        "amodal_bbox": amodal_bbox,
        "visible_bbox" : visible_bbox,
        "bbox": amodal_bbox,  # COCO 포맷 BBox 계산
        "area": area,
        "amodal_area": float(np.sum(amodal_mask == 255)),
        "amodal_segm": amodal_segm,
        "segmentation": amodal_segm,  # Segmentation polygon 생성
        "visible_segm": visible_segm,
        "background_objs_segm": [],  # 기본값
        "occluder_segm": occluder_segm,
    }
    return annotation

def process_amodal_images_and_masks(cucumber_image_path, 
                                    leaf_cropped_image_path, 
                                    cucumber_mask_path, 
                                    save_dir, 
                                    mask_save_dir, 
                                    coco_json, 
                                    global_image_id, 
                                    global_annotation_id, 
                                    position, 
                                    occlusion_ratio, 
                                    ):

    # 1. 오이 마스크 불러오기
    print("오이 이미지 합성 시작...")
    # 2. 오이 이미지에 잎 이미지를 합성하고 저장
    merged_image_path, amodal_mask, leaf_mask  = synthesize_two_images(cucumber_image_path, 
                                                                       cucumber_mask_path, 
                                                                       leaf_cropped_image_path, 
                                                                       position, 
                                                                       occlusion_ratio, 
                                                                       save_dir, 
                                                                       global_image_id)

    print("Modal 마스크 생성 시작...")
    # 3. Modal 마스크 생성 및 겹치는 부분 (가림) 정보 계산
    modal_mask, overlap_mask = generate_save_masks(
        amodal_mask, leaf_mask, os.path.basename(merged_image_path), mask_save_dir)
    
    print("COCO Format 데이터 생성 시작...")
    # 5. 공통 Annotation 생성
    cucumber_annotation = generate_annotation(
        amodal_mask=amodal_mask,
        modal_mask = modal_mask,
        global_id=global_annotation_id,
        image_id=global_image_id,
        category_id=1,
        occluder_segm=mask_to_polygon(leaf_mask)
    )
    coco_json["annotations"].append(cucumber_annotation)
    global_annotation_id += 1

    # 6. 잎 Annotation 생성
    leaf_annotation = generate_annotation(
        amodal_mask=leaf_mask,
        modal_mask = None,
        global_id=global_annotation_id,
        image_id=global_image_id,
        category_id=2,  # 잎 클래스 ID
    )
    coco_json["annotations"].append(leaf_annotation)
    global_annotation_id += 1
    
    # 7. 이미지 정보 추가
    image_info = {
        "id": global_image_id,
        "width": int(amodal_mask.shape[1]),
        "height": int(amodal_mask.shape[0]),
        "file_name": os.path.basename(merged_image_path),
    }
    coco_json["images"].append(image_info)

    # ID 증가
    global_image_id += 1

    # 9. 시각화
    #visualize_merged_amodal_and_modal(cv2.imread(merged_image_path, cv2.IMREAD_UNCHANGED), amodal_mask, modal_mask)

    return coco_json, global_image_id, global_annotation_id

### hypter parameters

In [5]:
''' input'''
cucumber_images_dir = INPUT_PATHS["cucumber_images_dir"]
cucumber_masks_dir = INPUT_PATHS["cucumber_masks_dir"]
leaf_cropped_dir = INPUT_PATHS["leaf_cropped_dir"]

''' output dir'''
save_dir = OUTPUT_PATHS["save_dir"]
mask_save_dir = OUTPUT_PATHS["mask_save_dir"]
json_dir = OUTPUT_PATHS["json_dir"]

'''paramters '''

image_index_start = HYPERPARAMETERS["image_index_start"]
sample_limit = HYPERPARAMETERS["saple_limit"]  # 생성할 샘플 수
sample_count = 0   # 현재 생성된 샘플 수
occlusion_ratio = HYPERPARAMETERS["occlusion_ratio"]  # 잎이 오이를 얼마나 가리는지 비율
position = HYPERPARAMETERS["position"]  # 잎이 오이를 어디에 위치할지

ensure_directories_exist([save_dir, mask_save_dir, json_dir])


디렉터리 생성됨: /home/knuvi/Desktop/song/occlusion-mask-generation/data/synthesis/amodal_images_resizing_test
디렉터리 생성됨: /home/knuvi/Desktop/song/occlusion-mask-generation/data/synthesis/modal_masks_resizing_test
디렉터리 생성됨: /home/knuvi/Desktop/song/occlusion-mask-generation/data/synthesis/amodal_info_resizing_test


#### Multiple Generation Code

In [ ]:
# 특정 클래스 마스크만 선택 (클래스 0: 오이)
def get_cucumber_masks(mask_dir, image_name):
    cucumber_masks = []
    for mask_file in os.listdir(mask_dir):
        # 파일명이 이미지명과 매칭되고 클래스가 0인 마스크만 선택
        if mask_file.startswith(image_name) and '_0_' in mask_file:
            cucumber_masks.append(os.path.join(mask_dir, mask_file))
    return cucumber_masks

# cucumber 이미지 파일 불러오기
cucumber_image_paths = get_image_paths_from_folder(cucumber_images_dir)

# COCO JSON 초기화
coco_json = initialize_coco_json()

global_image_id, global_annotation_id = 0,0
# 각 cucumber 이미지에 대해 마스크와 잎 합성
for cucumber_image_path in cucumber_image_paths:
    # 이미지 이름에서 확장자를 제거하여 기본 이미지명을 가져옴
    image_name = os.path.splitext(os.path.basename(cucumber_image_path))[0]
    
    # 해당 이미지의 오이 마스크들 가져오기 (클래스가 0인 것만)
    cucumber_mask_paths = get_cucumber_masks(cucumber_masks_dir, image_name)

    if len(cucumber_mask_paths) == 0:
        print(f"오이 마스크가 없습니다: {image_name}")
        continue

    # 잎 이미지 경로 선택 (임의의 하나 또는 여러 잎을 사용할 수 있음)
    leaf_cropped_image_paths = get_image_paths_from_folder(leaf_cropped_dir)
    print(f'{image_name}에 대한 {len(cucumber_mask_paths)}개의 마스크 확보...')
    
    # 오이 마스크와 잎 이미지 합성 처리
    for cucumber_mask_path in cucumber_mask_paths:
        print(f'{len(leaf_cropped_image_paths)} 개의 잎 이미지와 합성 시작...')

        for leaf_cropped_image_path in leaf_cropped_image_paths:

            if sample_count >= sample_limit:
                print(f"{sample_limit}개의 샘플 생성 완료.")
                break  # 안쪽 루프 종료

            # 오이와 잎 이미지를 합성하는 함수 호출
            coco_json,  global_image_id, global_annotation_id = process_amodal_images_and_masks(
                cucumber_image_path=cucumber_image_path,
                leaf_cropped_image_path=leaf_cropped_image_path,
                cucumber_mask_path=cucumber_mask_path,
                save_dir=save_dir,
                mask_save_dir=mask_save_dir,
                coco_json=coco_json,
                global_image_id= global_image_id,
                global_annotation_id= global_annotation_id,
                position=position,
                occlusion_ratio=occlusion_ratio,
            )
            sample_count += 1  # 샘플 생성 수 증가

        if sample_count >= sample_limit:
            break  # 바깥쪽 루프 종료

# 최종 COCO JSON 저장
output_json_path = os.path.join(json_dir, "dataset.json")
save_coco_json(coco_json, output_json_path)

오이 마스크가 없습니다: oi_347_1
오이 마스크가 없습니다: oi_398_1
oi_685_1에 대한 1개의 마스크 확보...
211 개의 잎 이미지와 합성 시작...
오이 이미지 합성 시작...
Processing leaf image: cropped_oi_224_1_1_00.png
Debug: Cucumber Area: 419170, Overlap Area: 226868, Leaf Area: 750000, Current Ratio: 0.5412
Debug: Cucumber Area: 419170, Overlap Area: 220698, Leaf Area: 720300, Current Ratio: 0.5265
Target ratio achieved with current ratio: 0.5265 after 1 iterations.
Leaves resized to target ratio.
Mask 저장됨: /home/knuvi/Desktop/song/occlusion-mask-generation/data/synthesis/amodal_images_resizing_test/oi_685_1_merged_000000.png
Modal 마스크 생성 시작...
Mask 저장됨: /home/knuvi/Desktop/song/occlusion-mask-generation/data/synthesis/modal_masks_resizing_test/oi_685_1_merged_000000_amodal_mask.png
Mask 저장됨: /home/knuvi/Desktop/song/occlusion-mask-generation/data/synthesis/modal_masks_resizing_test/oi_685_1_merged_000000_modal_mask.png
Mask 저장됨: /home/knuvi/Desktop/song/occlusion-mask-generation/data/synthesis/modal_masks_resizing_test/oi_685_1_merged_000